# Homework

The idea of this homework is to create a a LLM based application(RAG, smart prompt engineering, etc) by utilizing LLMOps tools like LangChain, Llamaindex and Langfuse.


### Requirements


In [1]:
# Install the langchain and langchain-community packages
!pip install langchain langchain-community boto3 langfuse faiss-cpu PyMuPDF

In [2]:
# Importing all the neccessary modules/libraries
import os

from langchain.chains.router import MultiRetrievalQAChain
from langchain.document_loaders import PyMuPDFLoader
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_aws import ChatBedrock
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.vectorstores import FAISS
from langfuse.callback import CallbackHandler

### Credentials, Loader, Chunker and Prompt Template

In [3]:
# Defining the configuration
REGION_NAME = "us-east-1"
CREDENTIALS_PROFILE_NAME = "MLEngineers"

PUBLIC_KEY = "pk-lf-c18a4846-e103-4db9-8739-328f70bb3b42" # e.g. "pk-1234567890abcdef"
SECRET_KEY = "sk-lf-a0258b40-6177-4c1e-93dd-243224f1b25d" # e.g. "sk-1234567890abcdef"
HOST = "https://cloud.langfuse.com"

EMBEDDER_MODEL_ID = "amazon.titan-embed-text-v2:0"
EMBEDDER_MODEL_KWARGS = {
    "dimensions": 512,
    "normalize": True
}

LLM_MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0" # anthropic.claude-3-haiku-20240307-v1:0 or anthropic.claude-3-sonnet-20240229-v1:0 or anthropic.claude-v2:1
LLM_MODEL_KWARGS = {
    "max_tokens": 4096,
    "temperature": 0.1,
    "top_p": 1,
    "top_k": 250,
    "stop_sequences": ["\n\nHuman"]
}

CHUNK_SIZE = 2000
CHUNK_OVERLAP = 100

DATA_PATHS = [
    "cooking_manuals/bimby.pdf",
    "cooking_manuals/gordon.pdf" 
]

INSTRUCTION_VECTOR_STORE_PATH = "./instruction_vector_database/"
RECIPE_VECTOR_STORE_PATH = "./recipe_vector_database/"

SEARCH_TYPE = "similarity"
RETRIEVER_KWARGS = {
    "k": 5
}

INPUT_KEY = "question"
INPUT_VARIABLES = ["context", "question"]

# Inside in the prompt template, you can play with the system's persona, the context, the history, and the question.
PROMPT_TEMPLATE = """
System: You are an helpful, respectful and honest assistant that cooking recipes, either by hand or using a TM6.
Always answer as helpfully as possible, while being safe.
Please ensure that your responses are socially unbiased and positive in nature.
When addressing the user, always base your responses on the context provided if its available.
If you are unsure about the answer, please let the user know.
If the user asks something that is not related to Cooking or TM6, please let the user know.
Human:
----------
<context>
{context}
</context>
----------
<question>
{question}
</question>
----------
Assistant:
"""

# Creating the memory and the prompt template
prompt = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=INPUT_VARIABLES
)

In [4]:
# Defining the chunker
splitter = RecursiveCharacterTextSplitter(
chunk_size=CHUNK_SIZE,
chunk_overlap=CHUNK_OVERLAP
)

In [5]:
# Creating chunks from the documents
global_chunks = []
for data_path in DATA_PATHS:
    loader = PyMuPDFLoader(os.path.join(os.getcwd(), data_path))
    docs = loader.load()
    chunks = splitter.split_documents(docs)
    global_chunks.append(chunks)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_community/document_loaders/parsers/pdf.py:299: UserWarning: Warning: Empty content on page 0 of document /Users/sanchabbarroso/Desktop/Internship/LLMOPS/cooking_manuals/gordon.pdf
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_community/document_loaders/parsers/pdf.py:299: UserWarning: Warning: Empty content on page 3 of document /Users/sanchabbarroso/Desktop/Internship/LLMOPS/cooking_manuals/gordon.pdf
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_community/document_loaders/parsers/pdf.py:299: UserWarning: Warning: Empty content on page 9 of document /Users/sanchabbarroso/Desktop/Internship/LLMOPS/cooking_manuals/gordon.pdf
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_community/document_loaders/parsers/pdf.py:299: Us

### Embedder

In [6]:
# Creating the embedder
embedder = BedrockEmbeddings(
    model_id=EMBEDDER_MODEL_ID,
    model_kwargs=EMBEDDER_MODEL_KWARGS,
    region_name=REGION_NAME,
    #credentials_profile_name=CREDENTIALS_PROFILE_NAME
)

/var/folders/j4/5cdm72g53yq6frsslklccrq40000gn/T/ipykernel_8056/592713195.py:2: LangChainDeprecationWarning: The class `BedrockEmbeddings` was deprecated in LangChain 0.2.11 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import BedrockEmbeddings``.
  embedder = BedrockEmbeddings(


### Vector Store (Database)

In [7]:
# Creating the vector store
instruction_vector_store = FAISS.from_documents(documents=global_chunks[0], embedding=embedder)
recipe_vector_store = FAISS.from_documents(documents=global_chunks[1], embedding=embedder)

instruction_vector_store.save_local(INSTRUCTION_VECTOR_STORE_PATH)
recipe_vector_store.save_local(RECIPE_VECTOR_STORE_PATH)

### LLM

In [8]:
# Creating the LLM and Embedder models
llm = ChatBedrock(
    region_name=REGION_NAME,
    model_id=LLM_MODEL_ID,
    model_kwargs=LLM_MODEL_KWARGS)
    #credentials_profile_name=CREDENTIALS_PROFILE_NAME,)

### MultiRetriever

In [9]:
# Loading the vector store and creating retriever
instruction_vector_store = FAISS.load_local(INSTRUCTION_VECTOR_STORE_PATH, embeddings=embedder, allow_dangerous_deserialization=True)
recipe_vector_store = FAISS.load_local(RECIPE_VECTOR_STORE_PATH, embeddings=embedder, allow_dangerous_deserialization=True)

instruction_retriever = instruction_vector_store.as_retriever(search_type=SEARCH_TYPE, **RETRIEVER_KWARGS)
recipe_retriever = recipe_vector_store.as_retriever(search_type=SEARCH_TYPE, **RETRIEVER_KWARGS)

retriever_infos = [
    {
        "name": "instructions", 
        "description": "Good for answering questions about TM6's safety instructions, how to use, settings and cleaning maintenance.", 
        "retriever": instruction_retriever,
        "prompt": prompt,
    },
    {
        "name": "recipes", 
        "description": "Good for answering questions Gordon Ramsay's recipes and cooking advices.",
        "retriever": recipe_retriever,
        "prompt": prompt,
    },
]

### Langfuse

In [10]:
# Creating the callback handler
langfuse_callback= CallbackHandler(
        public_key=PUBLIC_KEY,
        secret_key=SECRET_KEY,
        host=HOST,
    )

### MultiRetrievalQA chain

In [16]:
# Creating the Chain for usage
multi_chain = MultiRetrievalQAChain.from_retrievers(
            llm=llm,
            retriever_infos=retriever_infos,
            default_retriever=instruction_retriever,
            verbose=True)

response = multi_chain.invoke("How can I make a Soufflé just like Gordon Ramsay?", config={"callbacks": [langfuse_callback]})



> Entering new MultiRetrievalQAChain chain...
recipes: {'query': 'How can I make a Soufflé just like Gordon Ramsay?'}
> Finished chain.


In [17]:
print(response["result"])

Here are some tips for making soufflés like Gordon Ramsay:

1. Use a double layer of softened butter brushed in upward strokes inside the ramekins. This helps the soufflés rise evenly. Dust the butter with grated chocolate, flour, or other coatings suited to your flavor.

2. Whip the egg whites until they hold stiff, glossy peaks. This incorporates maximum air for a good rise. Stabilize them with a few drops of lemon juice or cream of tartar.

3. Fold the whipped egg whites into the base in two additions - first loosen the base by folding in 1/3 of the whites, then gently fold in the remaining whites with a light hand to keep the air incorporated.

4. Fill the ramekins to the top and run your finger around the inside edge to help them rise up straight. 

5. Bake at a high temperature (around 400°F/200°C) so they puff up quickly before drying out.

6. Have your oven preheated and ingredients at room temp so the soufflé batter doesn't lose volume before baking.

7. Consider making the ba

In [18]:
langfuse_callback.auth_check()

True

In [19]:
response = multi_chain.invoke("What components of a TM6 work when making a soufflé?", config={"callbacks": [langfuse_callback]})
print(response["result"])



> Entering new MultiRetrievalQAChain chain...
instructions: {'query': 'What components of the TM6 are used when making a soufflé?'}
> Finished chain.
When making a soufflé with the Thermomix TM6, the main components used would be:

- The stainless steel mixing bowl - This is where you would prepare the soufflé batter by whipping the egg whites and folding in the other ingredients.

- The butterfly whisk - This attachment is used for whipping egg whites to stiff peaks, which is crucial for getting the soufflé to rise properly.

- The measuring cup - For measuring out ingredients precisely.

- The spatula - Useful for scraping down the sides of the bowl and folding the whipped egg whites into the other ingredients gently.

- The varoma steamer - Soufflés are typically baked in the oven, but the varoma could potentially be used to steam them if adapting a recipe.

The mixing bowl, whisk, and spatula are the core components needed to prepare the light, airy soufflé batter properly in the

In [20]:
response = multi_chain.invoke("What would be the easiest recipe to cook by hand?", config={"callbacks": [langfuse_callback]})
print(response["result"])



> Entering new MultiRetrievalQAChain chain...
recipes: {'query': 'What would be the easiest recipe to cook by hand?'}
> Finished chain.
Based on the context provided, which discusses making cooking simple and convenient for everyday meals, some of the easiest recipes to cook by hand would be:

1. The bolognese sauce mentioned that will be ready before the pasta finishes boiling. Quick pasta sauces like this are great easy options.

2. The fishcakes that can be assembled from tins and jars. Using pre-cooked or canned ingredients cuts down on preparation time.

3. The chili hotdog described as being "in another league" compared to regular hotdogs. Dressing up simple items like hotdogs can make for an easy but tasty meal.

4. Baked beans on toast, which is mentioned as the go-to quick meal even for professional chefs after a long day of cooking.

5. Simple dishes like omelettes, pasta with tomato sauce, or risotto that can be easily customized with different add-ins as suggested in the 

In [21]:
response = multi_chain.invoke("What would be the easiest recipe to cook using a TM6?", config={"callbacks": [langfuse_callback]})
print(response["result"])



> Entering new MultiRetrievalQAChain chain...
recipes: {'query': 'What would be the easiest recipe to cook using a Thermomix TM6?'}
> Finished chain.
Based on the context provided, which focuses on quick and easy everyday cooking, a good recipe to try with the Thermomix TM6 would be something simple and fast like a soup or sauce.

Some easy Thermomix TM6 recipe ideas that could work well:

- Tomato Soup - Just add canned tomatoes, broth, and any desired seasonings/aromatics and blend until smooth.

- Pesto - Blend basil, pine nuts, garlic, parmesan, and olive oil for a quick fresh pesto sauce.

- Hummus - Cook chickpeas briefly then blend with tahini, lemon juice, garlic, and olive oil for smooth hummus.

- Salsa - Roughly chop tomatoes, onions, garlic, cilantro and blend briefly for a chunky salsa.

- Nut Butter - Blend roasted nuts until they release their oils and turn into a creamy nut butter.

The Thermomix makes these types of blended soups, dips, sauces and spreads extremely e